In [7]:
import openai
import os
import requests
import json
from concurrent.futures import ThreadPoolExecutor

# Add your own OpenAI API key
from dotenv import load_dotenv

from pathlib import Path

load_dotenv()

openai.api_key = os.getenv("AZURE_OPENAI_KEY")
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT") # your endpoint should look like the following https://YOUR_RESOURCE_NAME.openai.azure.com/
openai.api_type = 'azure'
openai.api_version = '2023-07-01-preview' # this may change in the future

In [8]:
source_text = Path("ct7_text.txt").read_text()
print(source_text)



Aravind Sridharan       
Awesome.
All right, everybody.
Why?
Perhaps turn my camera on.

Gervasi, Diana   joined the meeting

Aravind Sridharan       
How's everybody doing?

Masduki, Issy joined the meeting

Aravind Sridharan       
It's been a while, so you had a chalk talk.

Smati, Wided  TN joined the meeting

Aravind Sridharan       
OK, who here did some of the reading put a thumbs up?
It's OK if you did it, but who here did some of the reading for this week, or even skimmed through it, put a thumbs up if you did.

Masduki, Issy left the meeting

Aravind Sridharan       
OK, not a lot of you.
Alright, so that's fine.
But what do you guesses are allowed?
So today we're going to talk a little bit about dashboards and as a precursor to that, we'll actually talk about.

Smith, Jennifer joined the meeting

Aravind Sridharan       
Talk a little bit about.
We'll talk a little bit about workspaces right as well.
So personal define a dashboard and then we'll get it.
We'll talk about wo

In [9]:
def split_text(text, max_length=1024):
    chunks = []
    words = text.split()
    current_chunk = ""
    for word in words:
        if len(current_chunk) + len(word) < max_length:
            current_chunk += f" {word}"
        else:
            chunks.append(current_chunk.strip())
            current_chunk = f"{word}"
    if current_chunk:
        chunks.append(current_chunk.strip())
    return chunks

In [10]:
def generate_summary(text):
    input_chunks = split_text(text)
    output_chunks = []
    for chunk in input_chunks:
        response = openai.ChatCompletion.create(
            deployment_id="joept-35T-16K",
            messages =[ 
                {"role": "system", "content": "This text is a chunked summary of a transcript. Create a coherent summary based on the text. Avoid repeating names and do not mention when participants join or leave the meeting"},
                {"role": "user", "content": chunk}
            ],
            temperature=0.5,
            max_tokens=1024,
        )
        summary = response['choices'][0]['message']['content']
        output_chunks.append(summary)

    concat_text= " ".join(output_chunks)
    print(concat_text)
    return concat_text

In [11]:
def summary_of_summaries(first_summary_text):
    first_summary_text = generate_summary(first_summary_text)
    response = openai.ChatCompletion.create(
        deployment_id="joept-4",
        messages =[ 
            {"role": "system", "content": "This is a transcript from a class on Power BI. Extract the key points from the provided text, group them together by topic and display each topic group first with the topic in bold and then the summary of the topic below. Avoid repeating names and do not mention when participants join or leave the meeting"},
            {"role": "user", "content": first_summary_text}
        ],
        temperature=0.7,
        max_tokens=1500,
        
    )
    summary_of_summary = response['choices'][0]['message']['content']
    return summary_of_summary

In [12]:
responses = summary_of_summaries(source_text)
print(responses)

RateLimitError: Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2023-07-01-preview have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 34 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.